In [88]:
import pandas as pd
from collections import Counter
from sklearn import preprocessing
import numpy as np
import math
from scipy import stats

from sklearn import preprocessing
from sklearn.tree import DecisionTreeRegressor

### 1. Gather Data

In [2]:
def readtheTrainData(fname):
    return pd.read_csv(fname)

train_data = readtheTrainData('TrainData_PA.csv')
train_data_before_clean = train_data.copy()

### 2. Assess Data : Inspecting Data for Quality and Tidiness Issues
#### 2.1 Quality Issues : Issues with content - missing, duplicate or incorrect data. a.k.a Dirty data 
* 2.1.a Completeness : *"Are there any rows, columns or cells missing ?"*
  * Zipcode, county, address & property_type has the missing values
####   
* 2.1.b Validity : *"Does the data comply to the data schema like duplicate patient id or zip code being < 5 digits or float data type?"*
  * zipcode is float type
  * longitude and lattitude has to be represented in 3 dimentional space (x,y,z). So that digits can be used as the numerical value.
####   
  
* 2.1.c Accuracy : *"Wrong data that is valid. like hieght = 300 inches; it still complies to the standard i.e. inches but data is in accurate."*
  * Census_MedianIncome = 0 may be an outlier
####   
  
* 2.1.d Consistency : *"Both valid and accurate but inconsistent. state = california and CA"*
  * lotsize does not clearly indicate its metric of calculation.
####    


#### 2.2 Tidiness Issues: Issues with structure - untidy or messy data
* 2.2.a Each observation is a row
  * There are 50+ redundant data samples.
######   
* 2.2.b Each variable is a column
  * State is an obsolete column as it has only one value
  * property_type is categorical
  * garage, pool, fireplace, patio are binary
  * time is ordinal (2016.25, 2015.75, 2015.5, 2015.25)
######  
* 2.2.c Each observational unit is a table
  * 'time' variable has introduced redundancy in the dataset
######   

In [3]:
train_data

,county,city,zipcode,address,state,rent,latitude,longitude,cemetery_dist_miles,nationalhighway_miles,...,lotsize,Census_MedianIncome,CollegeGrads,WhiteCollar,Schools,Unemployment,EmploymentDiversity,Census_Vacancy,Crime_Rate,HomePrice
0,NaN,WEXFORD,NaN,266 Clematis Dr Allegheny County,PA,2400,40.618200,-80.077600,1.019586,0.206222,...,4086.388045,54476.09,21.0,66.57,48.3,5.10,3.48,3.42,2.400000,158051
1,NaN,WHITEHALL,NaN,2310 N 1st Ave Lehigh County,PA,995,40.649906,-75.478940,1.019586,0.206222,...,2247.513425,54476.09,21.0,66.57,48.3,5.10,3.48,3.42,2.400000,158051
2,NaN,WHITEHALL,NaN,3338 St Stephens Ln Lehigh County,PA,1740,40.646282,-75.510056,1.019586,0.206222,...,3109.741302,54476.09,21.0,66.57,48.3,5.10,3.48,3.42,2.400000,158051
3,NaN,WAYNESBORO,NaN,97 W Main St Franklin County,PA,675,39.756992,-77.579704,1.019586,0.206222,...,2349.673126,54476.09,21.0,66.57,48.3,5.10,3.48,3.42,2.400000,158051
4,NaN,QUAKERTOWN,NaN,200 E Broad St Bucks County,PA,1300,40.441177,-75.332540,1.019586,0.206222,...,2043.194023,54476.09,21.0,66.57,48.3,5.10,3.48,3.42,2.400000,158051
5,NaN,WAYNESBORO,NaN,407 Viewpoint Way Franklin County,PA,1025,39.766594,-77.582490,1.019586,0.206222,...,2635.720289,54476.09,21.0,66.57,48.3,5.10,3.48,3.42,2.400000,158051
6,NaN,WAYNESBORO,NaN,403 Viewpoint Way Franklin County,PA,1025,39.766580,-77.582450,1.019586,0.206222,...,2635.720289,54476.09,21.0,66.57,48.3,5.10,3.48,3.42,2.400000,158051
7,NaN,WAYNESBORO,NaN,240 Crown Ct Franklin County,PA,1025,39.759663,-77.575090,1.019586,0.206222,...,2635.720289,54476.09,21.0,66.57,48.3,5.10,3.48,3.42,2.400000,158051
8,NaN,WAYNESBORO,NaN,101 S Potomac St Franklin County,PA,795,39.755530,-77.581154,1.019586,0.206222,...,2451.832827,54476.09,21.0,66.57,48.3,5.10,3.48,3.42,2.400000,158051
9,NaN,UPPER DARBY,NaN,446 Woodcliffe Rd Delaware County,PA,925,39.958010,-75.264300,1.019586,0.206222,...,2051.366799,54476.09,21.0,66.57,48.3,5.10,3.48,3.42,2.400000,158051


In [4]:
for c in train_data.columns:
    print(train_data[c].describe())
    #input("Hit Enter : ")

count            18177
unique              61
top       PHILADELPHIA
freq              4970
Name: county, dtype: object
count            18203
unique             674
top       PHILADELPHIA
freq              4970
Name: city, dtype: object
count    18177.000000
mean     18030.505364
std       1482.025500
min      15001.000000
25%      17111.000000
50%      18944.000000
75%      19129.000000
max      19611.000000
Name: zipcode, dtype: float64
count                                    17496
unique                                   14579
top       Center City East Philadelphia County
freq                                        51
Name: address, dtype: object
count     18203
unique        1
top          PA
freq      18203
Name: state, dtype: object
count    18203.000000
mean      1601.734165
std        914.730775
min        385.000000
25%       1000.000000
50%       1400.000000
75%       1900.000000
max      16000.000000
Name: rent, dtype: float64
count    18203.000000
mean        40.087378
s

In [5]:
train_data_explore = train_data.describe(include='all')
print("Number of columns : ", train_data_explore.iloc[0].shape[0])
temp_df = pd.DataFrame(train_data_explore.iloc[0],columns=['count'])
print("features with NA Attributes : ", train_data.columns[train_data.isna().any()].tolist())
temp_df.sort_values('count').head(10)

Number of columns :  40
features with NA Attributes :  ['county', 'zipcode', 'address', 'property_type']


,count
property_type,15515
address,17496
county,18177
zipcode,18177
sqft,18203
garage,18203
yearbuilt,18203
pool,18203
fireplace,18203
patio,18203


In [6]:
train_data[train_data.columns[0:10]].describe(include='all')

,county,city,zipcode,address,state,rent,latitude,longitude,cemetery_dist_miles,nationalhighway_miles
count,18177,18203,18177.000000,17496,18203,18203.000000,18203.000000,18203.000000,18203.000000,18203.000000
unique,61,674,NaN,14579,1,NaN,NaN,NaN,NaN,NaN
top,PHILADELPHIA,PHILADELPHIA,NaN,Center City East Philadelphia County,PA,NaN,NaN,NaN,NaN,NaN
freq,4970,4970,NaN,51,18203,NaN,NaN,NaN,NaN,NaN
mean,NaN,NaN,18030.505364,NaN,NaN,1601.734165,40.087378,-77.267575,1.210770,0.292568
std,NaN,NaN,1482.025500,NaN,NaN,914.730775,1.578798,5.905950,0.936564,0.363775
min,NaN,NaN,15001.000000,NaN,NaN,385.000000,-11.968575,-149.900278,0.023038,0.000035
25%,NaN,NaN,17111.000000,NaN,NaN,1000.000000,39.967467,-77.012524,0.821590,0.145526
50%,NaN,NaN,18944.000000,NaN,NaN,1400.000000,40.130577,-75.477834,1.019586,0.206222
75%,NaN,NaN,19129.000000,NaN,NaN,1900.000000,40.444644,-75.185723,1.229043,0.283572


In [7]:
train_data[train_data.columns[10:20]].describe(include='all')

,railline_miles,starbucks_miles,walmart_miles,hospital_miles,physician_dist_miles,dentist_dist_miles,opt_dist_miles,vet_dist_miles,farmers_miles,time
count,18203.000000,18203.000000,18203.000000,18203.000000,18203.000000,18203.000000,18203.000000,18203.000000,18203.000000,18203.000000
mean,1.041623,2.043848,3.522699,2.270811,0.351478,0.708954,0.891707,12.368730,1.762844,2015.577913
std,1.529849,3.122275,1.758302,2.439700,0.538380,0.965186,1.065061,17.399292,2.154279,0.344428
min,0.000794,0.013197,0.118545,0.041096,0.000087,0.000157,0.000157,0.013511,0.014231,2015.250000
25%,0.470385,0.986498,2.719653,1.180311,0.170815,0.361726,0.495559,4.851441,0.751314,2015.250000
50%,0.629888,1.348776,3.326397,1.584675,0.229126,0.472933,0.651244,7.323725,1.094678,2015.500000
75%,0.862587,1.809722,3.864574,2.223802,0.300576,0.612327,0.858610,10.268145,1.709571,2015.750000
max,28.306430,61.349250,22.611844,32.433629,10.825859,19.823016,22.761643,141.184122,35.603985,2016.250000


In [8]:
train_data[train_data.columns[20:30]].describe(include='all')

,bed,bath,halfbath,sqft,property_type,garage,yearbuilt,pool,fireplace,patio
count,18203.000000,18203.000000,18203.000000,18203.000000,15515,18203.000000,18203.000000,18203.000000,18203.000000,18203.000000
unique,NaN,NaN,NaN,NaN,5,NaN,NaN,NaN,NaN,NaN
top,NaN,NaN,NaN,NaN,Condo,NaN,NaN,NaN,NaN,NaN
freq,NaN,NaN,NaN,NaN,7418,NaN,NaN,NaN,NaN,NaN
mean,2.766247,1.623743,1.027358,1536.611218,NaN,0.292424,1959.078284,0.011262,0.207603,0.197055
std,1.440967,0.855703,0.212040,826.928419,NaN,0.454889,23.153752,0.105526,0.405602,0.397785
min,1.000000,1.000000,1.000000,35.000000,NaN,0.000000,1800.000000,0.000000,0.000000,0.000000
25%,2.000000,1.000000,1.000000,1000.000000,NaN,0.000000,1960.000000,0.000000,0.000000,0.000000
50%,3.000000,1.000000,1.000000,1342.000000,NaN,0.000000,1960.000000,0.000000,0.000000,0.000000
75%,3.000000,2.000000,1.000000,1846.500000,NaN,1.000000,1960.000000,0.000000,0.000000,0.000000


In [9]:
train_data[train_data.columns[30:]].describe(include='all')

,lotsize,Census_MedianIncome,CollegeGrads,WhiteCollar,Schools,Unemployment,EmploymentDiversity,Census_Vacancy,Crime_Rate,HomePrice
count,18203.000000,18203.000000,18203.000000,18203.000000,18203.000000,18203.000000,18203.000000,18203.000000,18203.000000,1.820300e+04
mean,3683.627327,57076.140365,24.763198,67.419613,46.932605,5.182363,3.438426,4.887957,6.385568,1.836020e+05
std,3199.469032,23873.065621,15.978667,14.839700,26.840908,0.956217,0.150475,4.431504,7.310517,1.877200e+05
min,1.000000,0.000000,1.000000,15.090000,1.000000,2.970000,2.350000,0.000000,0.000000,2.452200e+04
25%,1724.000000,41040.665000,13.000000,56.200000,22.500000,4.500000,3.370000,1.320000,1.468825,1.537940e+05
50%,2872.730796,54476.090000,21.000000,66.570000,48.100000,5.100000,3.480000,3.420000,2.400000,1.580510e+05
75%,4290.707448,70379.790000,35.000000,79.340000,68.000000,6.000000,3.540000,7.040000,9.136104,1.613435e+05
max,19341.000000,198425.570000,72.000000,95.520000,99.000000,10.000000,3.670000,31.000000,39.159639,8.628600e+06


##### Let us focus on below from the visual observation on the column descriptions.
* Sqft = 35 
* CollegeGrads = 1 
* Schools = 1
* Census_Vacancy = 0
* Crime_Rate = 0

In [10]:
print("How many properties have sqft < 1000 :",train_data_before_clean[train_data_before_clean.sqft < 1000].shape)
print("Property type for sqft < 1000 : ", train_data_before_clean[train_data_before_clean.sqft < 1000].property_type.unique())
print("Property type for sqft < 500 : ", train_data_before_clean[train_data_before_clean.sqft < 500].property_type.unique())
print("Property type for sqft < 375 : ", train_data_before_clean[train_data_before_clean.sqft < 375].property_type.unique())
print("Property type for sqft < 300 : ", train_data_before_clean[train_data_before_clean.sqft < 300].property_type.unique())
print("Property type for sqft < 280 : ", train_data_before_clean[train_data_before_clean.sqft < 280].property_type.unique())

print("Property type for sqft < 100 : ", train_data_before_clean[train_data_before_clean.sqft < 100].property_type.unique())
print("Number of properties per Yearbuilt when sqft < 280 : ",Counter(train_data_before_clean[train_data_before_clean.sqft < 280][['bed','rent','sqft','yearbuilt','HomePrice']].yearbuilt.sort_values()))

How many properties have sqft < 1000 : (4539, 40)
Property type for sqft < 1000 :  ['Condo' nan 'SFR' 'Duplex']
Property type for sqft < 500 :  ['Condo' nan 'SFR' 'Duplex']
Property type for sqft < 375 :  ['Condo' nan 'SFR' 'Duplex']
Property type for sqft < 300 :  ['Condo' nan 'SFR' 'Duplex']
Property type for sqft < 280 :  ['Condo' nan 'SFR']
Property type for sqft < 100 :  ['Condo']
Number of properties per Yearbuilt when sqft < 280 :  Counter({1960: 29, 1945: 2, 1800: 1, 2007: 1})


In [11]:
train_data_before_clean[['yearbuilt','rent']].groupby('yearbuilt').\
        agg('count').rename(index=str, columns={'rent':'num of properties'}).\
        sort_values('num of properties',ascending=False).reset_index().head(10)

,yearbuilt,num of properties
0,1960,12967
1,1900,334
2,1920,224
3,1925,197
4,1915,189
5,1950,171
6,1970,155
7,2015,144
8,1930,131
9,2014,123


##### It appears as if post WWII , homecoming soldiers were given quaters and perhaps due to sharp demand they might had adjust a lot.
##### My view is that sqft = 35 ==> 7x5 area is not a outlier.

In [12]:
Counter(train_data_before_clean.CollegeGrads.sort_values(ascending=True))

Counter({1.0: 43,
         2.0: 263,
         3.0: 71,
         4.0: 277,
         5.0: 670,
         6.0: 388,
         7.0: 288,
         8.0: 448,
         9.0: 380,
         10.0: 450,
         11.0: 434,
         11.5: 1,
         12.0: 572,
         13.0: 931,
         14.0: 646,
         15.0: 517,
         16.0: 751,
         17.0: 489,
         18.0: 373,
         19.0: 704,
         20.0: 367,
         21.0: 535,
         22.0: 298,
         23.0: 337,
         24.0: 243,
         25.0: 264,
         26.0: 291,
         27.0: 312,
         28.0: 389,
         29.0: 166,
         30.0: 88,
         31.0: 606,
         32.0: 329,
         33.0: 298,
         34.0: 357,
         35.0: 254,
         36.0: 459,
         37.0: 273,
         38.0: 432,
         39.0: 201,
         40.0: 53,
         41.0: 352,
         42.0: 196,
         43.0: 104,
         44.0: 37,
         45.0: 145,
         46.0: 193,
         47.0: 149,
         48.0: 94,
         49.0: 40,
         50.0: 66,

##### It appears that there may be CollegeGrads == 1; perhaps it may be military soldiers or farmers area.

In [13]:
Counter(train_data_before_clean.Schools.sort_values(ascending=True))

Counter({1.0: 62,
         1.3: 50,
         1.8: 34,
         2.4: 29,
         2.6: 2,
         2.7: 3,
         2.9: 40,
         3.3: 277,
         3.6: 6,
         3.7: 25,
         3.8: 3,
         3.9: 6,
         4.0: 24,
         4.1: 22,
         4.2: 38,
         4.3: 7,
         4.4: 3,
         4.6: 14,
         4.9: 126,
         5.0: 3,
         5.1: 4,
         5.2: 23,
         5.5: 50,
         5.8: 7,
         6.0: 31,
         6.2: 37,
         6.4: 65,
         6.5: 16,
         6.7: 8,
         6.8: 47,
         6.9: 27,
         7.1: 2,
         7.4: 6,
         7.6: 69,
         7.7: 36,
         7.8: 13,
         8.0: 2,
         8.2: 27,
         8.3: 2,
         8.6: 17,
         8.7: 47,
         8.9: 28,
         9.0: 23,
         9.1: 10,
         9.9: 2,
         10.0: 30,
         10.1: 43,
         10.2: 69,
         10.3: 105,
         10.4: 116,
         10.7: 2,
         10.8: 10,
         10.9: 5,
         11.1: 11,
         11.3: 36,
         11.4:

##### It appears that there may be School == 1; perhaps it may be military soldiers or farmers area.

In [14]:
train_data_before_clean[train_data_before_clean.Census_Vacancy == 0][['rent','HomePrice','yearbuilt']].\
groupby('yearbuilt').agg('count').sort_values(by='rent',ascending=False).rename(index=str,columns={'rent':'count'}).\
reset_index()[['yearbuilt','count']].head()

,yearbuilt,count
0,1960,58
1,1900,2
2,1921,2
3,1950,2
4,2014,2


##### It appears that there may be Census_Vacancy == 0; perhaps it may be military soldiers or farmers area.

In [15]:
train_data_before_clean['rent_per_bed'] = train_data_before_clean.rent/ train_data_before_clean.bed

train_data_before_clean['rent_per_bed'].describe()

count    18203.000000
mean       664.040346
std        382.407154
min        100.000000
25%        416.666667
50%        550.000000
75%        785.000000
max       3950.000000
Name: rent_per_bed, dtype: float64

In [16]:
Counter(train_data_before_clean[train_data_before_clean.Crime_Rate == 0]['yearbuilt'])

Counter({1970: 3,
         1960: 122,
         2016: 1,
         1981: 1,
         2005: 1,
         2008: 3,
         2010: 1,
         2004: 3,
         2007: 1,
         2001: 1,
         1979: 2,
         1977: 1,
         1999: 1,
         1991: 2,
         2006: 2,
         1870: 3,
         2015: 3,
         1837: 1,
         1852: 1,
         1987: 1,
         1986: 1,
         1909: 1,
         1875: 1,
         2000: 1,
         1990: 2,
         1971: 1,
         1962: 1,
         1996: 1,
         1954: 1,
         1950: 1,
         2002: 1,
         1988: 2,
         1890: 1,
         1929: 1,
         1900: 3,
         1910: 2,
         1930: 1,
         1920: 2,
         1985: 1,
         1976: 1,
         1948: 1})

In [17]:
train_data_before_clean[train_data_before_clean.Crime_Rate == 0].property_type.unique()

array(['SFR', 'Condo', 'Duplex', nan], dtype=object)

##### It is again possible that it may be an farmer or military population. It may also happen that high rent place where securities are available like Quadplex and Triplex

In [18]:
train_data_before_clean.Census_MedianIncome.describe()

count     18203.000000
mean      57076.140365
std       23873.065621
min           0.000000
25%       41040.665000
50%       54476.090000
75%       70379.790000
max      198425.570000
Name: Census_MedianIncome, dtype: float64

In [19]:
def printSkewness(df):
    for c in list(df.select_dtypes(include=[np.number])):
        if df[c].skew() < -1 or df[c].skew() > 1:
            print("Skewness of",c,"is : ",df[c].skew()," and kurtosis : ",df[c].kurtosis())
printSkewness(train_data_before_clean)

Skewness of rent is :  3.3911691179387486  and kurtosis :  23.946087081275245
Skewness of latitude is :  -7.377510560987899  and kurtosis :  121.33405915002778
Skewness of longitude is :  -6.039359592984169  and kurtosis :  42.61646856026069
Skewness of cemetery_dist_miles is :  3.6306801808444065  and kurtosis :  18.9060074977957
Skewness of nationalhighway_miles is :  5.069967671920754  and kurtosis :  39.01139732042393
Skewness of railline_miles is :  5.645942204220113  and kurtosis :  54.54493569656745
Skewness of starbucks_miles is :  7.5884687838318055  and kurtosis :  85.09366875632581
Skewness of walmart_miles is :  2.3347694041664875  and kurtosis :  11.405607865813925
Skewness of hospital_miles is :  3.854622402708077  and kurtosis :  21.527097922521413
Skewness of physician_dist_miles is :  6.238265378612465  and kurtosis :  54.822025807746826
Skewness of dentist_dist_miles is :  5.201221007225711  and kurtosis :  39.692131623932696
Skewness of opt_dist_miles is :  5.5804542

##### let us see the feasibility of dropping the lotsize by searching if there is a high co relation feature to it.

In [20]:
def printCorr (df, lowerLimit = .7):
    corr_df = df.corr()

    for c in list(corr_df):
        temp_df = corr_df[(corr_df[c].abs() > .7) & (corr_df[c].abs() < 1) ]
        if not temp_df.empty:
            print(temp_df[c], "\n\n")
printCorr(train_data_before_clean)

dentist_dist_miles    0.741940
opt_dist_miles        0.721298
Name: physician_dist_miles, dtype: float64 


physician_dist_miles    0.741940
opt_dist_miles          0.801435
Name: dentist_dist_miles, dtype: float64 


physician_dist_miles    0.721298
dentist_dist_miles      0.801435
Name: opt_dist_miles, dtype: float64 


WhiteCollar    0.916561
Name: CollegeGrads, dtype: float64 


CollegeGrads    0.916561
Name: WhiteCollar, dtype: float64 




##### Shall we drop CollegeGrads , physician_dist_miles   &  opt_dist_miles  ??

# 3. Cleaning

##### https://datascience.stackexchange.com/questions/13567/ways-to-deal-with-longitude-latitude-feature
Lat long coordinates have a problem that they are 2 features that represent a three dimensional space. This means that the long coordinate goes all around, which means the two most extreme values are actually very close together. I've dealt with this problem a few times and what I do in this case is map them to x, y and z coordinates. This means close points in these 3 dimensions are also close in reality. Depending on the use case you can disregard the changes in height and map them to a perfect sphere. These features can then be standardized properly.

To clarify (summarised from the comments):

##### x = cos(lat) x cos(lon)
##### y = cos(lat) x sin(lon), 
##### z = sin(lat) 

##### we can map longitude and lattitude to a address [http://www.geonames.org/maps/us-reverse-geocoder.html#findNearestAddress]
##### vice versa can be done at https://www.latlong.net/
##### Therefore, we can ignore the features: Zipcode, county, address

In [65]:
def removeDuplicate(df):
    
    print('Initial df size', df.shape)
    
    print('Duplicates size', df[df.duplicated()].shape)
    
    print('NaN columns :', df.columns[df.isna().any()].tolist())
    
    df = df.fillna('NA')
    print('NaN columns (post fillna):', df.columns[df.isna().any()].tolist())
    print('Duplicates size (post fillna)', df[df.duplicated()].shape)
    
       
    key_columns = [x for x in list(df) if x != 'time']
    print("Duplicates without considering *time* feature : ", df[df.duplicated(key_columns)].shape)
    df = df.sort_values(by='time', ascending=False).reset_index(drop=True)
    
    df = df.drop_duplicates(key_columns,keep='first').reset_index(drop=True)
    df = df.drop_duplicates()
    
    print("Post duplicates removal data set size : ", df.shape)
    
    return df
    
df = removeDuplicate(train_data_before_clean)

print("\n")
printCorr(df)
printSkewness(df)

Initial df size (18203, 41)
Duplicates size (60, 41)
NaN columns : ['county', 'zipcode', 'address', 'property_type']
NaN columns (post fillna): []
Duplicates size (post fillna) (60, 41)
Duplicates without considering *time* feature :  (123, 41)
Post duplicates removal data set size :  (18080, 41)


dentist_dist_miles    0.742074
opt_dist_miles        0.719750
Name: physician_dist_miles, dtype: float64 


physician_dist_miles    0.742074
opt_dist_miles          0.802566
Name: dentist_dist_miles, dtype: float64 


physician_dist_miles    0.719750
dentist_dist_miles      0.802566
Name: opt_dist_miles, dtype: float64 


WhiteCollar    0.916674
Name: CollegeGrads, dtype: float64 


CollegeGrads    0.916674
Name: WhiteCollar, dtype: float64 


Skewness of rent is :  3.2828188250586843  and kurtosis :  22.050933104666207
Skewness of latitude is :  -7.359591755027664  and kurtosis :  120.69225402676179
Skewness of longitude is :  -6.025876438056323  and kurtosis :  42.38006802690317
Skewness o

In [66]:
def removeOutlier(df):
    print("\nShape of the data set before to outlier removal : ", df.shape)
    df = df[df.Census_MedianIncome != 0]
    print("Shape of the data set after outlier removal : ", df.shape)
    print()
    
    return df
    
removeOutlier(df).info()


Shape of the data set before to outlier removal :  (18080, 41)
Shape of the data set after outlier removal :  (18079, 41)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18079 entries, 0 to 18079
Data columns (total 41 columns):
county                   18079 non-null object
city                     18079 non-null object
zipcode                  18079 non-null object
address                  18079 non-null object
state                    18079 non-null object
rent                     18079 non-null int64
latitude                 18079 non-null float64
longitude                18079 non-null float64
cemetery_dist_miles      18079 non-null float64
nationalhighway_miles    18079 non-null float64
railline_miles           18079 non-null float64
starbucks_miles          18079 non-null float64
walmart_miles            18079 non-null float64
hospital_miles           18079 non-null float64
physician_dist_miles     18079 non-null float64
dentist_dist_miles       18079 non-null float64
opt_di

In [67]:
def dropCorrelatedFeatures(df):
    print("\nShape of the data set before dropping co-related features : ", df.shape)
    df = df.drop(['CollegeGrads','physician_dist_miles','opt_dist_miles'],axis=1)
    print("Shape of the data set post dropping co-related features : ", df.shape)
    print()
    return df
(dropCorrelatedFeatures(df)).info()


Shape of the data set before dropping co-related features :  (18080, 41)
Shape of the data set post dropping co-related features :  (18080, 38)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18080 entries, 0 to 18079
Data columns (total 38 columns):
county                   18080 non-null object
city                     18080 non-null object
zipcode                  18080 non-null object
address                  18080 non-null object
state                    18080 non-null object
rent                     18080 non-null int64
latitude                 18080 non-null float64
longitude                18080 non-null float64
cemetery_dist_miles      18080 non-null float64
nationalhighway_miles    18080 non-null float64
railline_miles           18080 non-null float64
starbucks_miles          18080 non-null float64
walmart_miles            18080 non-null float64
hospital_miles           18080 non-null float64
dentist_dist_miles       18080 non-null float64
vet_dist_miles           18080 n

In [68]:
def getDummiesForBooleanFeatures(df):
    print("\nShape of the data set before creating dummies : ", df.shape)
    
    df.garage = df.garage.apply(lambda x: 'garage_yes' if x == 0 else 'garage_no' )
    df.pool = df.pool.apply(lambda x: 'pool_yes' if x == 0 else 'pool_no' )
    df.fireplace = df.fireplace.apply(lambda x: 'fireplace_yes' if x == 0 else 'fireplace_no' )
    df.patio = df.patio.apply(lambda x: 'patio_yes' if x == 0 else 'patio_no' )
    
    categorical_columns = ['garage', 'pool', 'fireplace', 'patio']
    for c in categorical_columns:
        temp = pd.get_dummies(df[c])
        df[temp.columns] = temp
    df = df.drop(categorical_columns, axis=1)
    
    print("Shape of the data set post creating dummies : ", df.shape)
    print()
    return df
getDummiesForBooleanFeatures(df).info()


Shape of the data set before creating dummies :  (18080, 41)
Shape of the data set post creating dummies :  (18080, 45)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18080 entries, 0 to 18079
Data columns (total 45 columns):
county                   18080 non-null object
city                     18080 non-null object
zipcode                  18080 non-null object
address                  18080 non-null object
state                    18080 non-null object
rent                     18080 non-null int64
latitude                 18080 non-null float64
longitude                18080 non-null float64
cemetery_dist_miles      18080 non-null float64
nationalhighway_miles    18080 non-null float64
railline_miles           18080 non-null float64
starbucks_miles          18080 non-null float64
walmart_miles            18080 non-null float64
hospital_miles           18080 non-null float64
physician_dist_miles     18080 non-null float64
dentist_dist_miles       18080 non-null float64
opt_dist

In [70]:
def preProcessTheData(df):
    print("Shape of the data set before transforming : ", df.shape)
    
    #remove unncessary features
    df = df.drop(['zipcode','county','address','city','state'],axis=1)
    
    #transform longitude and latitude as discussed earlier
    df['x'] = df.latitude.apply(lambda x: math.cos(x)) * df.longitude.apply(lambda x : math.cos(x))
    df['y'] = df.latitude.apply(lambda x: math.cos(x)) * df.longitude.apply(lambda x : math.sin(x))
    df['z'] = df.latitude.apply(lambda x: math.sin(x))
    df = df.drop(['longitude','latitude'],axis=1)
    
    #get dummies for categorical features
    categorical_columns = ['time', 'property_type']
    for c in categorical_columns:
        temp = pd.get_dummies(df[c])
        df[temp.columns] = temp
    df = df.drop(categorical_columns, axis=1)
    
    print("Shape of the data set after transforming : ", df.shape,"\n")
    return df

preProcessTheData(removeOutlier(removeDuplicate(readtheTrainData('TrainData_PA.csv')))).info()

Initial df size (18203, 40)
Duplicates size (60, 40)
NaN columns : ['county', 'zipcode', 'address', 'property_type']
NaN columns (post fillna): []
Duplicates size (post fillna) (60, 40)
Duplicates without considering *time* feature :  (123, 40)
Post duplicates removal data set size :  (18080, 40)

Shape of the data set before to outlier removal :  (18080, 40)
Shape of the data set after outlier removal :  (18079, 40)

Shape of the data set before transforming :  (18079, 40)
Shape of the data set after transforming :  (18079, 44) 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18079 entries, 0 to 18079
Data columns (total 44 columns):
rent                     18079 non-null int64
cemetery_dist_miles      18079 non-null float64
nationalhighway_miles    18079 non-null float64
railline_miles           18079 non-null float64
starbucks_miles          18079 non-null float64
walmart_miles            18079 non-null float64
hospital_miles           18079 non-null float64
physician_dist_miles

In [71]:
def scaleMinMax(df):
    print("\nShape of the data set before min max scaling : ",df.shape)
    minMax = preprocessing.MinMaxScaler(feature_range=(.0001, 1))
    np_scaled = minMax.fit_transform(df)
    df_scaled = pd.DataFrame(np_scaled,columns=df.columns)
    print("\nShape of the data set post min max scaling : ",df_scaled.shape)
    return df_scaled

#scaleMinMax(df[df.columns.isin(list(df.select_dtypes(include=[np.number])))])
scaleMinMax(df[list(df.select_dtypes(include=[np.number]))]).info()



Shape of the data set before min max scaling :  (18080, 39)

Shape of the data set post min max scaling :  (18080, 39)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18080 entries, 0 to 18079
Data columns (total 39 columns):
rent                     18080 non-null float64
latitude                 18080 non-null float64
longitude                18080 non-null float64
cemetery_dist_miles      18080 non-null float64
nationalhighway_miles    18080 non-null float64
railline_miles           18080 non-null float64
starbucks_miles          18080 non-null float64
walmart_miles            18080 non-null float64
hospital_miles           18080 non-null float64
physician_dist_miles     18080 non-null float64
dentist_dist_miles       18080 non-null float64
opt_dist_miles           18080 non-null float64
vet_dist_miles           18080 non-null float64
farmers_miles            18080 non-null float64
time                     18080 non-null float64
bed                      18080 non-null float64
bat

In [72]:
def boxCoxTranformation(df):
    
    #assuming that only numerical features are presented
    
    print("Shape of the dataset before transformation : ", df.shape)
    
    print("Ignoring the columns....",list(df.select_dtypes(exclude=[np.number])))
    temp_df = df[list(df.select_dtypes(include=[np.number]))]
    print("Performing column transformations for :", list(temp_df) )
    
    #converting to positive for boxcox
    df_min_max = scaleMinMax(temp_df)
    #print(df_min_max)
    df_new = pd.DataFrame()
    for c in list(df_min_max):
        #print(c,df_min_max[c])
        temp_col_ser = df_min_max[c]
        #print(df_min_max[c].sort_values()[:5])
        #print(stats.boxcox(temp_col_ser)[0])
        df_new[c] = stats.boxcox(df_min_max[c])[0]
    
    print("Shape of the dataset after transformation : ", df_new.shape)
    return df_new
boxCoxTranformation(removeOutlier(removeDuplicate(readtheTrainData('TrainData_PA.csv')))).info()

Initial df size (18203, 40)
Duplicates size (60, 40)
NaN columns : ['county', 'zipcode', 'address', 'property_type']
NaN columns (post fillna): []
Duplicates size (post fillna) (60, 40)
Duplicates without considering *time* feature :  (123, 40)
Post duplicates removal data set size :  (18080, 40)

Shape of the data set before to outlier removal :  (18080, 40)
Shape of the data set after outlier removal :  (18079, 40)

Shape of the dataset before transformation :  (18079, 40)
Ignoring the columns.... ['county', 'city', 'zipcode', 'address', 'state', 'property_type']
Performing column transformations for : ['rent', 'latitude', 'longitude', 'cemetery_dist_miles', 'nationalhighway_miles', 'railline_miles', 'starbucks_miles', 'walmart_miles', 'hospital_miles', 'physician_dist_miles', 'dentist_dist_miles', 'opt_dist_miles', 'vet_dist_miles', 'farmers_miles', 'time', 'bed', 'bath', 'halfbath', 'sqft', 'garage', 'yearbuilt', 'pool', 'fireplace', 'patio', 'lotsize', 'Census_MedianIncome', 'Coll

In [74]:
df =boxCoxTranformation(preProcessTheData(removeOutlier(removeDuplicate(readtheTrainData('TrainData_PA.csv')))))
printCorr(df)
printSkewness(df)
df.info()

Initial df size (18203, 40)
Duplicates size (60, 40)
NaN columns : ['county', 'zipcode', 'address', 'property_type']
NaN columns (post fillna): []
Duplicates size (post fillna) (60, 40)
Duplicates without considering *time* feature :  (123, 40)
Post duplicates removal data set size :  (18080, 40)

Shape of the data set before to outlier removal :  (18080, 40)
Shape of the data set after outlier removal :  (18079, 40)

Shape of the data set before transforming :  (18079, 40)
Shape of the data set after transforming :  (18079, 44) 

Shape of the dataset before transformation :  (18079, 44)
Ignoring the columns.... []
Performing column transformations for : ['rent', 'cemetery_dist_miles', 'nationalhighway_miles', 'railline_miles', 'starbucks_miles', 'walmart_miles', 'hospital_miles', 'physician_dist_miles', 'dentist_dist_miles', 'opt_dist_miles', 'vet_dist_miles', 'farmers_miles', 'bed', 'bath', 'halfbath', 'sqft', 'garage', 'yearbuilt', 'pool', 'fireplace', 'patio', 'lotsize', 'Census_Me

In [75]:
df.head()

,rent,cemetery_dist_miles,nationalhighway_miles,railline_miles,starbucks_miles,walmart_miles,hospital_miles,physician_dist_miles,dentist_dist_miles,opt_dist_miles,...,2015.25,2015.5,2015.75,2016.25,Condo,Duplex,NA,Quadplex,SFR,Triplex
0,-1.965950,-1.983336,-2.505237,-3.661331,-3.677427,-1.374147,-2.796557,-3.797105,-3.524654,-3.387719,...,-22.992479,-80.207641,-172.428898,0.0,0.000000,-50206.494484,-1093.425695,-5.692548e+96,-31.742351,-1.184771e+32
1,-2.964429,-1.983336,-2.505237,-3.661331,-3.677427,-1.374147,-2.796557,-3.797105,-3.524654,-3.387719,...,-22.992479,-80.207641,-172.428898,0.0,0.000000,-50206.494484,-1093.425695,-5.692548e+96,-31.742351,-1.184771e+32
2,-2.784630,-2.653594,-2.972199,-5.027029,-6.234656,-1.186219,-3.434071,-3.143872,-3.247478,-3.377358,...,-22.992479,-80.207641,-172.428898,0.0,-17.132913,-50206.494484,-1093.425695,-5.692548e+96,0.000000,-1.184771e+32
3,-3.196713,-1.922606,-2.460441,-3.930657,-2.609269,-1.277892,-2.062451,-1.301210,-1.810154,-1.955893,...,-22.992479,-80.207641,-172.428898,0.0,-17.132913,-50206.494484,-1093.425695,-5.692548e+96,0.000000,-1.184771e+32
4,-2.554276,-2.208123,-3.181549,-4.825136,-3.570923,-1.070731,-3.511698,-3.784503,-3.076798,-2.742492,...,-22.992479,-80.207641,-172.428898,0.0,0.000000,-50206.494484,-1093.425695,-5.692548e+96,-31.742351,-1.184771e+32


In [76]:
df.to_csv('TrainData_PA_cleansed',index=False)

In [324]:
def returnTrainTestSet(df,frac=.7,random_state=200):
    
    print("Input data set shape : ",df.shape)
    all_columns = set(df.columns)
    all_columns.discard('rent')
    feature_columns = list(all_columns)
    lable_column = 'rent'
    
    print("\nfeature_columns : ", feature_columns)
    print("lable_column : ", lable_column)
    
    df = scaleMinMax(df)
    
    df_train = df.sample(frac=frac,random_state=random_state)
    df_test = df.drop(df_train.index)
    print("\nTrain feature shape: ", df_train[feature_columns].shape, df_train[lable_column].shape)
    print("Test feature shape: ", df_test[feature_columns].shape, df_test[lable_column].shape)
    
    return df_train[feature_columns], df_train[lable_column], df_test[feature_columns], df_test[lable_column]
X_train, y_train, X_test, y_test = returnTrainTestSet(df)

Input data set shape :  (18079, 44)

feature_columns :  ['patio', 'Census_MedianIncome', 'pool', 'Duplex', 'physician_dist_miles', 'sqft', 'Condo', 'Unemployment', 'EmploymentDiversity', 'starbucks_miles', 'nationalhighway_miles', 'walmart_miles', 'cemetery_dist_miles', 'farmers_miles', 'bath', 'railline_miles', 'SFR', 'WhiteCollar', 'hospital_miles', 'halfbath', 'y', 'vet_dist_miles', 'bed', 'yearbuilt', 'opt_dist_miles', 'dentist_dist_miles', 'fireplace', 'Schools', 'garage', 'CollegeGrads', 'z', 2015.25, 2015.5, 2015.75, 2016.25, 'Triplex', 'Crime_Rate', 'Quadplex', 'NA', 'HomePrice', 'lotsize', 'Census_Vacancy', 'x']
lable_column :  rent

Shape of the data set before min max scaling :  (18079, 44)

Shape of the data set post min max scaling :  (18079, 44)

Train feature shape:  (12655, 43) (12655,)
Test feature shape:  (5424, 43) (5424,)


In [215]:
clf = DecisionTreeRegressor(max_depth=4)
clf.fit(X_train, y_train)
clf.score(X_test, y_test)

0.5814329756484303

##### Manual trial and error method resulted in following reading :
###### (max_depth=3) : 0.5494546459120873
###### (max_depth=4) : 0.5814329756484303
###### (max_depth=5) : 0.5697837122502597
###### (max_depth=6) : 0.566588549926632
###### (max_depth=7) : 0.5493435534445739
###### (max_depth=8) : 0.5341133112176326
###### (max_depth=9) : 0.5159846446868466

In [216]:
clf.feature_importances_

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.63556239,
       0.        , 0.        , 0.02961113, 0.        , 0.        ,
       0.00514878, 0.        , 0.08885215, 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.20063133,
       0.01677253, 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.02342168,
       0.        , 0.        , 0.        ])

In [319]:
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
clf = RandomForestRegressor(max_depth=4, n_estimators=500)
clf.fit(X_train, y_train)
clf.score(X_test, y_test)


0.6245113157711104

##### Manual trial and error gave following results:
###### (max_depth=None) : 0.6392922870733752
###### (max_depth=3) : 0.5721141694756593
###### (max_depth=4) : 0.6269848309321449
###### (max_depth=5) : 0.6069183978911982
###### (max_depth=6) : 0.6173980247154297
###### (max_depth=7) : 
###### (max_depth=8) : 
###### (max_depth=9) : 

clf = AdaBoostRegressor()
clf.fit(X_train, y_train)
clf.score(X_test, y_test)

In [ ]:
from sklearn.model_selection import ShuffleSplit, cross_val_score

X,y = returnXy(train_data_cleansed_1)
cv = ShuffleSplit(n_splits=50, test_size=0.3, random_state=1986)
scores = cross_val_score(clf, X, y, cv=cv) 

In [ ]:
import numpy as np
np.average(scores)

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
#arr = pca.fit_transform(train_data_cleansed_1)
ttt = train_data_cleansed_1.drop('rent',axis=1)
#arr = pca.fit_transform(ttt)
arr = pca.fit_transform(X)

In [ ]:
df_plot = pd.DataFrame(arr)
df_plot.plot(0,1)

In [ ]:
import matplotlib.pyplot as plt
plt.scatter(df_plot[df_plot.columns[0]],df_plot[df_plot.columns[1]])
plt.show()

from sklearn.model_selection import GridSearchCV
#parameters = {'max_depth':[2,3,4,5,6,7,8,9,10],}# 'min_samples_split':[2, 75]}
parameters = {'max_depth':list(range(2,15)), 'min_samples_split':list(range(5,100,5))}
Dtr = DecisionTreeRegressor()
clf = GridSearchCV(Dtr, parameters, cv=5,scoring='r2')
clf.fit(X, y)

##### clf.get_params()

clf.cv_results_['mean_test_score']

clf.best_estimator_

In [ ]:
#clf = RandomForestRegressor(n_estimators=100,max_depth=8,min_samples_split=20)
#clf = RandomForestRegressor(n_estimators=100,max_depth=8,min_samples_split=75)
clf = RandomForestRegressor(n_estimators=50,max_depth=9,min_samples_split=75)
clf.fit(X_train, y_train)
clf.score(X_test, y_test)

In [ ]:
cv = ShuffleSplit(n_splits=10, test_size=0.3, random_state=50)
scores = cross_val_score(clf, X, y, cv=cv) 
scores

In [ ]:
#prepare test data
def testDataPreprocessing(df):
    df = df.drop(['zipcode','county','address','city','state'],axis=1)
    df['x'] = df.latitude.apply(lambda x: math.cos(x)) * \
                                                        df.longitude.apply(lambda x : math.cos(x))
    df['y'] = df.latitude.apply(lambda x: math.cos(x)) * \
                                                        df.longitude.apply(lambda x : math.sin(x))
    df['z'] = df.latitude.apply(lambda x: math.sin(x))
    df = df.drop(['longitude','latitude'],axis=1)
    
    df.garage = df.garage.apply(lambda x: 'garage_yes' if x == 0 else 'garage_no' )
    df.pool = df.pool.apply(lambda x: 'pool_yes' if x == 0 else 'pool_no' )
    df.fireplace = df.fireplace.apply(lambda x: 'fireplace_yes' if x == 0 else 'fireplace_no' )
    df.patio = df.patio.apply(lambda x: 'patio_yes' if x == 0 else 'patio_no' )
    categorical_columns = ['time', 'property_type', 'garage', 'pool', 'fireplace', 'patio']
    for c in categorical_columns:
        temp = pd.get_dummies(df[c])
        df[temp.columns] = temp
    df = df.drop(categorical_columns, axis=1)
    
    return returnXy(df)

print(clf)
test_X, test_y = testDataPreprocessing(test_data)
print(test_X.info())
clf.score(test_X, test_y)

In [ ]:
test_result = clf.predict(test_X)
test_result

In [334]:
def printStats(test_result,test_y):
    per_count=[]
    #print(test_result,test_y)
    test_y = np.array(test_y)
    for i in range(test_y.size):
        if test_y[i]:
            per_count.append(0)
        else:
            per_count.append(np.round(np.abs((test_result[i]- test_y[i])*100/test_y[i]),2))
    cnt = Counter(per_count)
    less_1 = less_2 = less_3 = less_4 = less_5 = 0
    for c in cnt:
        if c < 1:
            less_1 += cnt[c]
        if c < 2:
            less_2 += cnt[c]
        if c < 3:
            less_3 += cnt[c]
        if c < 4:
            less_4 += cnt[c]
        if c < 5 :
            less_5 += cnt[c]

    print(np.round((less_1*100/test_y.size),2),"% of properties within predicted rent within 1% of actual rent")
    print(np.round((less_2*100/test_y.size),2),"% of properties within predicted rent within 2% of actual rent")
    print(np.round((less_3*100/test_y.size),2),"% of properties within predicted rent within 3% of actual rent")
    print(np.round((less_4*100/test_y.size),2),"% of properties within predicted rent within 4% of actual rent")
    print(np.round((less_5*100/test_y.size),2),"% of properties within predicted rent within 5% of actual rent")
printStats(test_result,test_y)

NameError: name 'test_result' is not defined

In [320]:
'''sklearn.linear_model.SGDRegressor
sklearn.linear_model.Lasso 
sklearn.linear_model.ElasticNet
sklearn.linear_model.Ridge
sklearn.svm.SVR
sklearn.ensemble.AdaBoostRegressor
sklearn.ensemble.GradientBoostingRegressor
'''
from sklearn.linear_model import SGDRegressor, LassoCV, ElasticNetCV, RidgeCV
from sklearn.svm import SVR
from sklearn.ensemble import AdaBoostRegressor, GradientBoostingRegressor

clf_s = [SGDRegressor(), LassoCV(), ElasticNetCV(), RidgeCV(), AdaBoostRegressor(),GradientBoostingRegressor()]#, SVR(kernel='linear')]
for c in clf_s:
    c.fit(X_train, y_train)
    print(c,c.score(X_test, y_test))

C:\Users\padmaraj.bhat\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDRegressor'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


SGDRegressor(alpha=0.0001, average=False, epsilon=0.1, eta0=0.01,
       fit_intercept=True, l1_ratio=0.15, learning_rate='invscaling',
       loss='squared_loss', max_iter=None, n_iter=None, penalty='l2',
       power_t=0.25, random_state=None, shuffle=True, tol=None, verbose=0,
       warm_start=False) 0.5433932377997952
LassoCV(alphas=None, copy_X=True, cv=None, eps=0.001, fit_intercept=True,
    max_iter=1000, n_alphas=100, n_jobs=1, normalize=False, positive=False,
    precompute='auto', random_state=None, selection='cyclic', tol=0.0001,
    verbose=False) 0.6440124642835834
ElasticNetCV(alphas=None, copy_X=True, cv=None, eps=0.001, fit_intercept=True,
       l1_ratio=0.5, max_iter=1000, n_alphas=100, n_jobs=1,
       normalize=False, positive=False, precompute='auto',
       random_state=None, selection='cyclic', tol=0.0001, verbose=0) 0.6468827000289981
RidgeCV(alphas=(0.1, 1.0, 10.0), cv=None, fit_intercept=True, gcv_mode=None,
    normalize=False, scoring=None, store_cv_values

In [321]:
clf_s[-1].score(test_X,test_y)

NameError: name 'test_X' is not defined

In [ ]:
printStats(clf_s[-1].predict(test_X),test_y)

In [ ]:
clf_g = GradientBoostingRegressor(max_depth=9, min_samples_split=75)
clf_g.fit(X_train,y_train)
printStats(clf_g.predict(test_X),test_y)


In [ ]:
clf_g = GradientBoostingRegressor(max_depth=8, min_samples_split=75)
clf_g.fit(X_train,y_train)
printStats(clf_g.predict(test_X),test_y)


In [ ]:
clf_g = GradientBoostingRegressor(max_depth=8, min_samples_split=50)
clf_g.fit(X_train,y_train)
printStats(clf_g.predict(test_X),test_y)


In [ ]:
clf_g = GradientBoostingRegressor(max_depth=9, min_samples_split=50)
clf_g.fit(X_train,y_train)
printStats(clf_g.predict(test_X),test_y)


In [ ]:
clf_g = GradientBoostingRegressor(n_estimators=500,max_depth=8, min_samples_split=50)
clf_g.fit(X_train,y_train)
printStats(clf_g.predict(test_X),test_y)

In [ ]:
clf_g = GradientBoostingRegressor(n_estimators=1500,max_depth=8, min_samples_split=50)
clf_g.fit(X_train,y_train)
printStats(clf_g.predict(test_X),test_y)

#### is best number of estimator between 500- 1500?

In [ ]:
from sklearn.model_selection import GridSearchCV

parameters = {'n_estimators':list(range(200,1500,100)),}# 'min_samples_split':[2, 75]} parameters = {'max_depth':list(range(2,15)), 'min_samples_split':list(range(5,100,5))} 
clf_g = GradientBoostingRegressor(max_depth=8, min_samples_split=50)
clf_grid_g = GridSearchCV(clf_g, parameters, cv=5,scoring='r2') 


In [ ]:
clf_grid_g.fit(X,y)

In [ ]:
clf_final = clf_grid_g.best_estimator_

In [ ]:
clf_grid_g.cv_results_['mean_test_score']

In [ ]:
clf_final.score(test_X,test_y)

In [ ]:
printStats(clf_final.predict(test_X),test_y)

##### Unfortunately, not conclusive, let us try XGBoost

In [335]:
from xgboost import XGBRegressor

my_model = XGBRegressor()#max_depth=8, n_estimators=500)
my_model.fit(X_train,y_train)
print(my_model.score(X_test,y_test))
printStats(my_model.predict(X_test),y_test)

0.7725393748835422
100.0 % of properties within predicted rent within 1% of actual rent
100.0 % of properties within predicted rent within 2% of actual rent
100.0 % of properties within predicted rent within 3% of actual rent
100.0 % of properties within predicted rent within 4% of actual rent
100.0 % of properties within predicted rent within 5% of actual rent


In [ ]:
printStats(my_model.predict(test_X),test_y)

3.13 % of properties within predicted rent within 1% of actual rent
5.73 % of properties within predicted rent within 2% of actual rent
8.53 % of properties within predicted rent within 3% of actual rent
11.88 % of properties within predicted rent within 4% of actual rent
14.88 % of properties within predicted rent within 5% of actual rent

In [ ]:
my_model_1 = XGBRegressor(n_estimators=1000, learning_rate=0.05)
my_model_1.fit(X_train, y_train, early_stopping_rounds=5, 
             eval_set=[(X_test, y_test)], verbose=False)

In [ ]:
printStats(my_model_1.predict(test_X),test_y)

In [ ]:
my_model_2 = XGBRegressor(max_depth=4, n_estimators=1000, learning_rate=0.05)
my_model_2.fit(X_train, y_train, early_stopping_rounds=5, 
             eval_set=[(X_test, y_test)], verbose=False)
printStats(my_model_2.predict(test_X),test_y)

In [ ]:
my_model_2 = XGBRegressor(max_depth=7, n_estimators=1000, learning_rate=0.05)
my_model_2.fit(X_train, y_train, early_stopping_rounds=5, 
             eval_set=[(X_test, y_test)], verbose=False)
printStats(my_model_2.predict(test_X),test_y)

In [ ]:
my_model_3 = XGBRegressor(max_depth=8, n_estimators=1000, learning_rate=0.05)
my_model_3.fit(X_train, y_train, early_stopping_rounds=5, 
             eval_set=[(X_test, y_test)], verbose=False)
printStats(my_model_3.predict(test_X),test_y)

In [ ]:
my_model_4 = XGBRegressor(max_depth=9, n_estimators=1000, learning_rate=0.05)
my_model_4.fit(X_train, y_train, early_stopping_rounds=5, 
             eval_set=[(X_test, y_test)], verbose=False)
printStats(my_model_4.predict(test_X),test_y)

In [ ]:
my_model_5 = XGBRegressor(max_depth=10, n_estimators=1000, learning_rate=0.05)
my_model_5.fit(X_train, y_train, early_stopping_rounds=5, 
             eval_set=[(X_test, y_test)], verbose=False)
printStats(my_model_5.predict(test_X),test_y)

In [ ]:
my_model_5 = XGBRegressor(max_depth=8, n_estimators=500, learning_rate=0.1)
#, eval_set=[(X_test, y_test)]
my_model_5.fit(X, y, verbose=True)
printStats(my_model_5.predict(test_X),test_y)

In [ ]:
my_model_5 = XGBRegressor(max_depth=9, n_estimators=500, learning_rate=0.1)
#, eval_set=[(X_test, y_test)]
my_model_5.fit(X, y, verbose=True)
printStats(my_model_5.predict(test_X),test_y)

In [ ]:
my_model_5

##### Let us see if the property type prediction helps in the prediction.

In [ ]:
final_predict_property_type = predict_property_type[predict_property_type.property_type.isna()]
train_predict_property_type = predict_property_type.dropna()
train_predict_property_type
print("Train Samples : ", train_predict_property_type.shape, " Test Samples : ",test_predict_property_type.shape )
print(train_predict_property_type.head(2))

In [ ]:
def returnXYforProperType(df):
    y = df['property_type']
    
    from sklearn import preprocessing
    le = preprocessing.LabelEncoder()
    le.fit(y)
    
    y= le.transform(y)
    
    X = df.copy()
    X = X.drop('property_type', axis=1)
    np_scaled = min_max_scaler.fit_transform(X)
    df_normalized = pd.DataFrame(np_scaled,columns=X.columns)
    return df_normalized,y

X_prop_train, y_prop_train = returnXYforProperType(train_predict_property_type)
#X_prop_predict, y_prop_predict = returnXYforProperType(final_predict_property_type)

'''clf_prop = DecisionTreeRegressor(max_depth=10, min_samples_split=50)
clf_prop.fit(X_prop_train[:-200],y_prop_train[:-200])
print(clf_prop.score(X_prop_train[-200:], y_prop_train[-200:])) #0.378002969525984'''

'''clf_rfr = RandomForestRegressor(n_estimators=500, max_depth=10, min_samples_split=50)
clf_rfr.fit(X_prop_train[:-200],y_prop_train[:-200])
clf_rfr.score(X_prop_train[-200:], y_prop_train[-200:])#0.3730836253361981'''

clf_xgbr = XGBRegressor(n_estimators=200, max_depth=10, min_samples_split=50)
clf_xgbr.fit(X_prop_train[:-200],y_prop_train[:-200])
clf_xgbr.score(X_prop_train[-200:], y_prop_train[-200:])#0.3730836253361981'''

##### It is not worth to impute based on prediction as the confidence of the prediction is not even 50%. We do not want to introduce outliers into our dataset. There is a reduction of 14.77% in dataset size due to ignoring samples with NaN values for 'property_type'.

In [ ]:
print("Reduction in the Dataset size due to rejecting samples of NaN values : ",np.round((train_data[train_data.property_type.isna()]).shape[0] * 100 / train_data.shape[0],2),"%")

In [ ]:
print(test_data[test_data.property_type.isna()].shape)
print(test_data.shape)
na_test_data = test_data[test_data.property_type.isna()]
na_train_X, na_train_y = testDataPreprocessing(na_test_data)
printStats(my_model_5.predict(na_train_X),na_train_y)

##### This is the serious problem !!!! we need to use mapper for this.

In [ ]:
print(test_data[test_data.property_type.isna()].shape)
print(test_data.shape)
na_test_data = test_data[test_data.property_type.isna()]
na_train_X, na_train_y = testDataPreprocessing(test_data)
print(na_train_X[test_data.property_type.isna()].shape, na_train_X.shape)
print(na_train_y[test_data.property_type.isna()].shape)
printStats(my_model_5.predict(na_train_X[test_data.property_type.isna()]),np.array(na_train_y[test_data.property_type.isna()]))

##### What does this mean is that sub sample of test data has lower scores. It may be lowering the overall output. Let us check if na values are removed from the test data; would it give a better output.

In [ ]:
print("Total number of entries is the test data set : ", test_data.shape[0])
print("Number of Nan entries in the test data set : ", test_data[test_data.property_type.isna()].shape[0])
print ("Difference : ",  test_data.shape[0] - test_data[test_data.property_type.isna()].shape[0] ,"\n")

na_test_data = test_data[test_data.property_type.isna()]
na_train_X, na_train_y = testDataPreprocessing(test_data)

na_train_X = na_train_X.drop(na_train_X[test_data.property_type.isna()].index)
print("Post ignoring the Nan entries, test data size of X: ", na_train_X.shape[0])

na_train_y = na_train_y.drop(na_train_y[test_data.property_type.isna()].index)
print("Post ignoring the Nan entries, test data size of y: ", na_train_y.shape[0], "\n")

printStats(my_model_5.predict(na_train_X),np.array(na_train_y))

The best ever result so far was :
* 3.28 % of properties within predicted rent within 1% of actual rent
* 6.18 % of properties within predicted rent within 2% of actual rent
* 9.35 % of properties within predicted rent within 3% of actual rent
* 12.08 % of properties within predicted rent within 4% of actual rent
* 14.88 % of properties within predicted rent within 5% of actual rent

##### However, this does not give any computational confidence of how important the missing values are !!!!! It is only post a valid impute operation we would know for sure how good the prediction would be.
##### Perhaps, in Exploratory Data Analysis we may try to visually represent the impact.